# LA County COVID-19 Cases by Neighborhoods

Use LA County's [countywide statistical areas](http://geohub.lacity.org/datasets/lacounty::countywide-statistical-areas-csa) and look at trends in cases and new cases by neighborhood.

In [1]:
import ipywidgets
import pandas as pd

import default_parameters 
import neighborhood_charts

from datetime import date
from IPython.display import Markdown 

yesterday_date = default_parameters.yesterday_date

start_date = date(2020, 4, 1)

In [2]:
df = neighborhood_charts.prep_data(start_date)

In [3]:
def top_5_bottom_5(df, yesterday_date):
    keep_cols = ["aggregate_region", "cases", "cases_per100k", "rank", "max_rank"]
    
    df = (df[df.date == yesterday_date][keep_cols]
          .assign(
             cases = df.cases.map("{:,}".format),
             rank = df["rank"].astype(int),
             max_rank = df.max_rank.astype(int),
             cases_per100k = df.cases_per100k.round(2).map("{:,.2f}".format),
          )
          .rename(columns = {"aggregate_region": "Neighborhood"})
          .sort_values("rank")
         )
    
    
    cols_to_show = ["Neighborhood", "cases", "cases_per100k"]
    
    top_5 = (df[df["rank"] <= 5][cols_to_show]
             .reset_index(drop=True)
            )
    
    bottom_5 = (df[(df["rank"] >= df["max_rank"] - 5)]
                .sort_values("rank", ascending=False)
                [cols_to_show]
                .reset_index(drop=True)
               )
    
    
    display(Markdown(
        "5 Neighborhoods with <strong>Worst Outbreak</strong> (ranked on cases per 100k)"
        )
    )
    
    display(top_5)
    
    display(Markdown(
        "5 Neighborhoods with <strong>Least Severe</strong> Outbreak (ranked on cases per 100k)"
        )
    )
    display(bottom_5)


In [4]:
top_5_bottom_5(df, yesterday_date)

5 Neighborhoods with <strong>Worst Outbreak</strong> (ranked on cases per 100k)

,Neighborhood,cases,cases_per100k
0,Castaic,"2,358","8,324.80"
1,Wholesale District,"2,914","7,630.07"
2,Vernon Central,"3,792","7,006.52"
3,Central,"2,740","7,006.42"
4,Pacoima,"5,322","6,657.49"


5 Neighborhoods with <strong>Least Severe</strong> Outbreak (ranked on cases per 100k)

,Neighborhood,cases,cases_per100k
0,La Canada Flintridge,238,591.26
1,Miracle Mile,299,877.63
2,Avalon,36,892.41
3,Rancho Palos Verdes,408,950.87
4,Santa Monica Mountains,318,"1,007.86"
5,Pacific Palisades,295,"1,062.41"


<strong><span style='color:f"#515252"'>The 25th and 75th percentile of cases and cases per 100k are calculated each day and drawn in gray.</span></strong>


<strong><span style='color:#1696D2'>The neighborhood's own trend is drawn in blue.</span></strong>


The percentiles provide context to whether a neighborhood's cases are relatively high or lower compared to other parts of the county. Data from the past 2 weeks are shaded in gray.

**Note:** Some neighborhoods report cases, but do not have population data. Rankings are calculated using cases per 100k to normalize across population differences. Neighborhoods without population data are excluded from the rankings.

In [5]:
# Make interactive chart
neigborhood_dropdown = ipywidgets.Dropdown(
    description="Neighborhood",
    style = {"description_width":"initial"},
    options=df.aggregate_region.unique()
)

output = ipywidgets.Output()

display(neigborhood_dropdown)
display(output)

def on_neighborhood_selection(*args):
    output.clear_output(wait=True)
    neighborhood = neigborhood_dropdown.value
    
    subset_df = df[df.aggregate_region == neighborhood]
    
    with output:
        neighborhood_charts.make_chart(subset_df, neighborhood)


on_neighborhood_selection()
neigborhood_dropdown.observe(on_neighborhood_selection, names="value")

Dropdown(description='Neighborhood', options=('Acton', 'Agoura Hills', 'Alhambra', 'Altadena', 'Arcadia', 'Arl…

Output()